In [64]:
import tldextract
import tld
import pandas as pd
from glob import glob

import pyspark.sql.functions as psf
import pyspark.sql.types as pst

In [139]:
%run ./spark-instance-gustavo.ipynb

SparkConf created
Started SparkSession
Spark version 3.5.0


In [143]:
clean_spark()

In [140]:
goscanner_df = spark.read.parquet(f"../dataset/catrin/ldap-dependency-goscanner.parquet")
goscanner_df.select(psf.explode_outer("leaf_data_names").alias("san")).distinct().coalesce(1).pandas_api().to_csv("../dataset/processing/leaf_data_names", index=False)

/usr/local/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [141]:
def fld(domain):
    if domain is None:
        return None
    domain = str(domain)
    try:
        tld.get_tld(domain, fail_silently=False, fix_protocol=True)
        return tld.get_fld(domain, fail_silently=False, fix_protocol=True)
    except tld.exceptions.TldBadUrl:
        return None
    except tld.exceptions.TldDomainNotFound:
        return None
    except ValueError:
        return None


def get_sld(domain):
    if domain is None:
        return None
    domain = str(domain)
    try:
        return tldextract.extract(domain).registered_domain
    except:
        return None


c = glob("../dataset/processing/leaf_data_names/*.csv")
san_pdf = pd.read_csv(c[0])
san_pdf["fld"] = san_pdf.san.apply(fld)
san_pdf["sld"] = san_pdf.san.apply(get_sld)

valid_names_pdf = san_pdf[san_pdf["fld"].notnull()]
additional_names_pdf = pd.DataFrame(valid_names_pdf["sld"].unique(), columns=["san"])
names_pdf = pd.concat([pd.DataFrame(valid_names_pdf["san"], columns=["san"]), additional_names_pdf], ignore_index=True).drop_duplicates(subset=["san"])
names_pdf = names_pdf[names_pdf["san"] != ""].sort_values("san", ascending=True)
names_pdf.to_csv("../dataset/processing/san.csv", index=False)